In [2]:
import cv2
import numpy as np
import mediapipe as mp
import pickle
import os

In [3]:
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(min_detection_confidence=0.7, min_tracking_confidence=0.7)

In [4]:
gestures_db = {}

In [7]:
db_filename = "gestures.pkl"
if os.path.exists(db_filename):
    with open(db_filename, "rb") as f:
        gestures_db = pickle.load(f)

def extract_hand_landmarks(image):
    """Extract hand landmarks from the image."""
    img_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = hands.process(img_rgb)
    
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            landmarks = []
            for lm in hand_landmarks.landmark:
                landmarks.append([lm.x, lm.y, lm.z])  # Normalize coordinates
            return np.array(landmarks).flatten()  # Convert to 1D array
    return None

In [8]:
def add_new_gesture(name, landmarks):
    """Store new hand gesture in the database."""
    gestures_db[name] = landmarks
    with open(db_filename, "wb") as f:
        pickle.dump(gestures_db, f)
    print(f"Gesture '{name}' saved successfully!")

def recognize_gesture(landmarks):
    """Recognize the closest matching gesture."""
    if not gestures_db:
        return "No gestures saved"
    
    min_distance = float("inf")
    recognized_gesture = None

    for gesture, stored_landmarks in gestures_db.items():
        distance = np.linalg.norm(landmarks - stored_landmarks)
        if distance < min_distance:
            min_distance = distance
            recognized_gesture = gesture
    
    return recognized_gesture if min_distance < 0.1 else "Unknown gesture"

In [9]:
cap = cv2.VideoCapture(0)

mode = "recognize"  # Default mode

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)  # Flip for better usability
    landmarks = extract_hand_landmarks(frame)
    
    if landmarks is not None:
        if mode == "recognize":
            gesture_name = recognize_gesture(landmarks)
            cv2.putText(frame, f"Gesture: {gesture_name}", (50, 50),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        elif mode == "add":
            gesture_name = input("Enter gesture name: ")
            add_new_gesture(gesture_name, landmarks)
            mode = "recognize"  # Switch back after adding

    cv2.imshow("Hand Gesture Recognition", frame)

    key = cv2.waitKey(1) & 0xFF
    if key == ord("q"):  # Quit
        break
    elif key == ord("n"):  # Switch to add mode
        mode = "add"

cap.release()
cv2.destroyAllWindows()

: 